<a href="https://colab.research.google.com/github/dburchill/ML_nano/blob/master/tfds01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install  -U tensorflow-datasets
!pip install  -U tensorflow-hub

Requirement already up-to-date: tensorflow-datasets in /usr/local/lib/python3.6/dist-packages (3.2.0)
Requirement already up-to-date: tensorflow-hub in /usr/local/lib/python3.6/dist-packages (0.8.0)


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', tf.keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

Using:
	• TensorFlow version: 2.2.0
	• tf.keras version: 2.3.0-tf
	• GPU device not found. Running on CPU


In [ ]:
train_split = 60
test_val_split = 20

#splits = tfds.Split.TRAIN.subsplit([60,20, 20])

(training_set,  test_set), dataset_info = tfds.load('cats_vs_dogs',split=['train', 'test'], as_supervised=True, with_info=True)

ValueError: ignored